LOADING THE DATASET

In [81]:
import pandas as pd
import ast

# Read all datasets
df_bangalore = pd.read_excel(r"C:\Users\vinoo\Downloads\bangalore_cars.xlsx")
df_delhi = pd.read_excel(r"C:\Users\vinoo\Downloads\delhi_cars.xlsx")
df_kolkata = pd.read_excel(r"C:\Users\vinoo\Downloads\kolkata_cars.xlsx")
df_hyderabad = pd.read_excel(r"C:\Users\vinoo\Downloads\hyderabad_cars.xlsx")
df_jaipur = pd.read_excel(r"C:\Users\vinoo\Downloads\jaipur_cars.xlsx")
df_chennai = pd.read_excel(r"C:\Users\vinoo\Downloads\chennai_cars.xlsx")


In [82]:
# Add city column to all datasets
df_bangalore.insert(0, "city", "Bangalore", True)
df_delhi.insert(0, "city", "Delhi", True)
df_kolkata.insert(0, "city", "Kolkata", True)
df_hyderabad.insert(0, "city", "Hyderabad", True)
df_jaipur.insert(0, "city", "Jaipur", True)
df_chennai.insert(0, "city", "Chennai", True)


In [83]:
# Concatenate all datasets
combined_df = pd.concat([df_bangalore, df_delhi, df_kolkata, df_hyderabad, df_jaipur, df_chennai], ignore_index=True)

In [84]:
# Rename columns
car_dekho = combined_df.rename(columns={
    'city': 'City',
    'new_car_detail': 'Details',
    'new_car_overview': 'Overview',
    'new_car_feature': 'Feature',
    'new_car_specs': 'Specs',
    'car_links': 'Car Links'
})

PREPROCESSING THE DATA

In [85]:
# Normalize the Details column
car_dekho['Details'] = car_dekho['Details'].apply(ast.literal_eval)
details_df = pd.json_normalize(car_dekho['Details'])
car_dekho = car_dekho.join(details_df).drop('Details', axis=1)

In [86]:
# Normalize the Overview column
car_dekho['Overview'] = car_dekho['Overview'].apply(ast.literal_eval)
overview_df = pd.json_normalize(car_dekho['Overview'])
car_dekho = car_dekho.join(overview_df, rsuffix='_overview').drop('Overview', axis=1)


In [87]:
# Flattening Overview: extracting data from 'top' and 'bottomData'
for index in car_dekho.index:
    if 'top' in car_dekho.columns and isinstance(car_dekho.at[index, 'top'], list):
        for entry in car_dekho.at[index, 'top']:
            if 'key' in entry and 'value' in entry:
                car_dekho.at[index, entry['key']] = entry['value']

    if 'bottomData' in car_dekho.columns and isinstance(car_dekho.at[index, 'bottomData'], list):
        for entry in car_dekho.at[index, 'bottomData']:
            car_dekho.at[index, entry['heading']] = entry['list']


In [88]:
# Drop 'top' and 'bottomData' columns for Overview
car_dekho = car_dekho.drop(columns=['top', 'bottomData'], errors='ignore')


In [89]:
# Normalize the Feature column
car_dekho['Feature'] = car_dekho['Feature'].apply(ast.literal_eval)
feature_df = pd.json_normalize(car_dekho['Feature'])
car_dekho = car_dekho.join(feature_df, rsuffix='_feature').drop('Feature', axis=1)

In [90]:
# Flattening Feature: extracting data from 'top' and 'data'
for index in car_dekho.index:
    if 'top' in car_dekho.columns and isinstance(car_dekho.at[index, 'top'], list):
        for entry in car_dekho.at[index, 'top']:
            if isinstance(entry, str):
                car_dekho.at[index, 'Feature_' + str(index)] = entry

    if 'data' in car_dekho.columns and isinstance(car_dekho.at[index, 'data'], list):
        for section in car_dekho.at[index, 'data']:
            if 'heading' in section and 'list' in section:
                for entry in section['list']:
                    if 'key' in entry and 'value' in entry:
                        car_dekho.at[index, entry['key']] = entry['value']

In [91]:
# Drop 'top' and 'data' columns for Feature
car_dekho = car_dekho.drop(columns=['top', 'data'], errors='ignore')


In [92]:
# Normalize the Specs column
car_dekho['Specs'] = car_dekho['Specs'].apply(ast.literal_eval)
specs_df = pd.json_normalize(car_dekho['Specs'])
car_dekho = car_dekho.join(specs_df, rsuffix='_specs').drop('Specs', axis=1)

In [93]:
# Flattening Specs: extracting data from 'top' and 'data'
for index in car_dekho.index:
    if 'top' in car_dekho.columns and isinstance(car_dekho.at[index, 'top'], list):
        for entry in car_dekho.at[index, 'top']:
            if 'key' in entry and 'value' in entry:
                car_dekho.at[index, entry['key']] = entry['value']

    if 'data' in car_dekho.columns and isinstance(car_dekho.at[index, 'data'], list):
        for section in car_dekho.at[index, 'data']:
            if 'heading' in section and 'list' in section:
                for entry in section['list']:
                    if 'key' in entry and 'value' in entry:
                        car_dekho.at[index, entry['key']] = entry['value']

In [94]:
# Drop 'top' and 'data' columns for Specs
car_dekho = car_dekho.drop(columns=['top', 'data'], errors='ignore')

In [95]:
# Display the final DataFrame
car_dekho.head(1)

,City,Car Links,it,ft,bt,km,transmission,ownerNo,owner,oem,...,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Wheel Size,Alloy Wheel Size,Ground Clearance Unladen
0,Bangalore,https://www.cardekho.com/used-car-details/used...,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,...,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,NaN,NaN,NaN


In [96]:
car_dekho.to_excel(r"C:\Users\vinoo\Downloads\structured_car_data.xlsx", index=False)

In [97]:
car_dekho = car_dekho[['City', 'bt', 'transmission', 'ownerNo' , 'oem' , 'model' ,'modelYear' , 'price' , 'Insurance Validity' , 'Fuel Type' , 'Seats' , 'Kms Driven' , 'Mileage']]

In [98]:
car_dekho.head(3)

,City,bt,transmission,ownerNo,oem,model,modelYear,price,Insurance Validity,Fuel Type,Seats,Kms Driven,Mileage
0,Bangalore,Hatchback,Manual,3,Maruti,Maruti Celerio,2015,₹ 4 Lakh,Third Party insurance,Petrol,5,"1,20,000 Kms",23.1 kmpl
1,Bangalore,SUV,Manual,2,Ford,Ford Ecosport,2018,₹ 8.11 Lakh,Comprehensive,Petrol,5,"32,706 Kms",17 kmpl
2,Bangalore,Hatchback,Manual,1,Tata,Tata Tiago,2018,₹ 5.85 Lakh,Comprehensive,Petrol,5,"11,949 Kms",23.84 kmpl


In [99]:
car_dekho.to_excel(r"C:\Users\vinoo\Downloads\final_carData.xlsx", index=False)

In [100]:
car_dekho.shape

(8369, 13)

In [101]:
car_dekho.columns = car_dekho.columns.str.lower()

In [102]:
car_dekho.rename(columns = {'bt':'body type'}, inplace = True)
car_dekho.rename(columns = {'oem':'brand'}, inplace = True)

In [103]:
car_dekho.isnull().sum()

city                    0
body type               0
transmission            0
ownerno                 0
brand                   0
model                   0
modelyear               0
price                   0
insurance validity      4
fuel type               0
seats                   5
kms driven              2
mileage               287
dtype: int64

In [104]:
car_dekho.dtypes

city                  object
body type             object
transmission          object
ownerno                int64
brand                 object
model                 object
modelyear              int64
price                 object
insurance validity    object
fuel type             object
seats                 object
kms driven            object
mileage               object
dtype: object

In [105]:
car_dekho['insurance validity'].fillna(car_dekho['insurance validity'].mode()[0], inplace=True )

C:\Users\vinoo\AppData\Local\Temp\ipykernel_226600\1407332355.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  car_dekho['insurance validity'].fillna(car_dekho['insurance validity'].mode()[0], inplace=True )


In [106]:
car_dekho['price']= car_dekho['price'].str.extract(r'(\d+\.?\d*)').astype(float)
# car_dekho['price']= car_dekho['price'].str.replace(r'[^0-9.]', '', regex= True)
car_dekho['price']= pd.to_numeric(car_dekho['price'], errors= 'coerce')


In [107]:
car_dekho['seats'] = pd.to_numeric(car_dekho['seats'], errors='coerce')
car_dekho['seats'].fillna(car_dekho['seats'].median(), inplace= True)
car_dekho['seats'] = car_dekho['seats'].astype(int)

C:\Users\vinoo\AppData\Local\Temp\ipykernel_226600\2618608655.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  car_dekho['seats'].fillna(car_dekho['seats'].median(), inplace= True)


In [108]:
car_dekho['kms driven'] = car_dekho['kms driven'].str.replace(r'[^\d.]', '', regex=True).astype(float)
car_dekho['kms driven']= pd.to_numeric(car_dekho['kms driven'], errors= 'coerce')
car_dekho['kms driven'].fillna(car_dekho['kms driven'].median(), inplace = True)

C:\Users\vinoo\AppData\Local\Temp\ipykernel_226600\3622564744.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  car_dekho['kms driven'].fillna(car_dekho['kms driven'].median(), inplace = True)


In [109]:
car_dekho['mileage'] = car_dekho['mileage'].str.replace('kmpl', '').str.replace('km/kg', '').str.strip().astype(float)
# car_dekho['mileage'] = car_dekho['mileage'].str.replace(r'[^\d.]', '', regex=True).astype(float)
car_dekho['mileage'] = pd.to_numeric(car_dekho['mileage'], errors='coerce')
car_dekho['mileage'].fillna(car_dekho['mileage'].median(), inplace=True)

C:\Users\vinoo\AppData\Local\Temp\ipykernel_226600\544250222.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  car_dekho['mileage'].fillna(car_dekho['mileage'].median(), inplace=True)


In [110]:
car_dekho.head(3)

,city,body type,transmission,ownerno,brand,model,modelyear,price,insurance validity,fuel type,seats,kms driven,mileage
0,Bangalore,Hatchback,Manual,3,Maruti,Maruti Celerio,2015,4.00,Third Party insurance,Petrol,5,120000.0,23.10
1,Bangalore,SUV,Manual,2,Ford,Ford Ecosport,2018,8.11,Comprehensive,Petrol,5,32706.0,17.00
2,Bangalore,Hatchback,Manual,1,Tata,Tata Tiago,2018,5.85,Comprehensive,Petrol,5,11949.0,23.84


In [111]:
# car_dekho['price']= car_dekho['price']*100000
# car_dekho

In [112]:
car_dekho.columns = car_dekho.columns.str.replace(' ','_')

In [113]:
car_dekho.isnull().sum()

city                  0
body_type             0
transmission          0
ownerno               0
brand                 0
model                 0
modelyear             0
price                 0
insurance_validity    0
fuel_type             0
seats                 0
kms_driven            0
mileage               0
dtype: int64

In [114]:
car_dekho.describe()

,ownerno,modelyear,price,seats,kms_driven,mileage
count,8369.000000,8369.000000,8369.000000,8369.000000,8.369000e+03,8369.00000
mean,1.360139,2016.503286,10.017363,5.203131,5.898714e+04,19.24365
std,0.641958,3.921465,12.621907,0.663573,7.405543e+04,3.96145
min,0.000000,1985.000000,1.000000,2.000000,1.010000e+02,7.08000
25%,1.000000,2014.000000,4.000000,5.000000,3.000000e+04,17.01000
50%,1.000000,2017.000000,6.190000,5.000000,5.370000e+04,18.90000
75%,2.000000,2019.000000,9.750000,5.000000,8.000000e+04,21.40000
max,5.000000,2023.000000,99.000000,10.000000,5.500000e+06,140.00000


In [115]:
print(car_dekho.transmission.value_counts())
print(car_dekho.insurance_validity.value_counts())
print(car_dekho.fuel_type.value_counts())

transmission
Manual       6052
Automatic    2317
Name: count, dtype: int64
insurance_validity
Third Party insurance    4059
Comprehensive            3150
Third Party               630
Zero Dep                  303
Not Available             223
1                           3
2                           1
Name: count, dtype: int64
fuel_type
Petrol      5554
Diesel      2673
CNG           98
Electric      28
LPG           16
Name: count, dtype: int64


In [116]:
car_dekho.to_csv(r"C:\Users\vinoo\OneDrive\文档\shared files\capstone_project3\new_cleaned_data.csv", index=False)